### SEM SAM2 Segmentation Pipeline (Kaggle)
This notebook was executed in the Kaggle environment  
using `/kaggle/input` datasets and saving outputs to `/kaggle/working`.

In [ ]:
import os, time

# Local working directory in Kaggle (to store models/outputs)
WORK_ROOT = "/kaggle/working/SEM_SAM2"
DIRS = {
    "root": WORK_ROOT,
    "models": f"{WORK_ROOT}/models",   # model weights/configs will be saved here
    "output": f"{WORK_ROOT}/output",   # overlay results and manifests will be saved here
}

for d in DIRS.values():
    os.makedirs(d, exist_ok=True)

# Where do the images come from?
INPUT_DIRS = ["/kaggle/input"]

# Root for relative paths in manifest (cosmetic, for path display)
MAP_ROOT = "/kaggle"

print("Images will be read from:", INPUT_DIRS)
print("Outputs will be written under:", DIRS["output"])


Images will be read from: ['/kaggle/input']
Outputs will be written under: /kaggle/working/SEM_SAM2/output


In [ ]:
!pip -q install opencv-python matplotlib supervision
!pip -q install git+https://github.com/facebookresearch/segment-anything-2.git

import torch, cv2, numpy as np, glob, os
print("CUDA available:", torch.cuda.is_available())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 5.5 MB/s eta 0:00:00:00:01


In [ ]:
!wget -qO- https://httpbin.org/ip | head -c 80


In [ ]:
import os, urllib.request

CFG_NAME = "sam2_hiera_l.yaml"
CKPT_NAME = "sam2_hiera_large.pt"

CFG_PATH  = os.path.join(DIRS["models"], CFG_NAME)
CKPT_PATH = os.path.join(DIRS["models"], CKPT_NAME)

CFG_URL  = "https://huggingface.co/spaces/SkalskiP/segment-anything-model-2/resolve/main/configs/sam2_hiera_l.yaml"
CKPT_URL = "https://huggingface.co/spaces/SkalskiP/segment-anything-model-2/resolve/main/checkpoints/sam2_hiera_large.pt"

os.makedirs(DIRS["models"], exist_ok=True)

def download(url, dst):
    print(f"Downloading -> {dst}")
    urllib.request.urlretrieve(url, dst)
    sz = os.path.getsize(dst) / (1024*1024)
    print(f"Saved {dst} ({sz:.1f} MB)")

if not os.path.exists(CFG_PATH):
    download(CFG_URL, CFG_PATH)
else:
    print("Config exists:", CFG_PATH)

if not os.path.exists(CKPT_PATH):
    download(CKPT_URL, CKPT_PATH)  # ~900MB
else:
    print("Weights exist:", CKPT_PATH)

print("Ready.")


In [ ]:
SAM2_PARAMS = dict(
    points_per_side=32,
    points_per_batch=64,
    pred_iou_thresh=0.7,
    stability_score_thresh=0.92,
    stability_score_offset=0.7,
    crop_n_layers=1,
    box_nms_thresh=0.7,
)

ALPHA = 0.45
MAX_IMAGES = None   
print("SAM2 params:", SAM2_PARAMS)


In [ ]:
import numpy as np, cv2

def to_rgb(img):
    if img.ndim == 2:
        return cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def colorize_instances_sam2(masks, h, w):
    overlay = np.zeros((h, w, 3), dtype=np.uint8)
    n = max(len(masks), 1)
    for i, m in enumerate(masks):
        seg = m["segmentation"]
        hue = int(180 * i / n)
        color_hsv = np.uint8([[[hue, 200, 255]]])
        color_bgr = cv2.cvtColor(color_hsv, cv2.COLOR_HSV2BGR)[0,0,:]
        color_rgb = color_bgr[::-1]
        overlay[seg] = color_rgb
    return overlay

def blend_overlay(rgb, overlay_rgb, alpha=0.45):
    return (rgb * (1 - alpha) + overlay_rgb * alpha).astype(np.uint8)


In [ ]:
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
import sam2, shutil

device = "cuda" if torch.cuda.is_available() else "cpu"

CFG_NAME_FOR_HYDRA = "sam2_hiera_l.yaml"  # Hydra expects the name, not the full path

# If the config file isn’t packaged—copy it into the package directory once
SAM2_PKG_DIR = os.path.dirname(sam2.__file__)
PKG_CFG = os.path.join(SAM2_PKG_DIR, CFG_NAME_FOR_HYDRA)
if (not os.path.exists(PKG_CFG)) and os.path.exists(CFG_PATH):
    try:
        shutil.copy(CFG_PATH, PKG_CFG)
        print("Copied YAML into package:", PKG_CFG)
    except Exception as e:
        print("Note:", e)

sam2_model = build_sam2(CFG_NAME_FOR_HYDRA, CKPT_PATH, device=device)
mask_generator = SAM2AutomaticMaskGenerator(model=sam2_model, **SAM2_PARAMS)

print("SAM2 loaded on", device)


In [ ]:
import os, glob, csv, time, cv2, random, shutil
from datetime import datetime

# Collect files from all datasets in the list, recursively, for all relevant extensions
PATTERNS = [
    "**/*.png","**/*.jpg","**/*.jpeg","**/*.tif","**/*.tiff","**/*.bmp",
    "**/*.PNG","**/*.JPG","**/*.JPEG","**/*.TIF","**/*.TIFF","**/*.BMP",
]
all_paths = []
for base in INPUT_DIRS:
    for pat in PATTERNS:
        all_paths += glob.glob(os.path.join(base, pat), recursive=True)

all_paths = sorted(set(all_paths))

# Optional limit
try:
    if MAX_IMAGES is not None:
        all_paths = all_paths[:MAX_IMAGES]
except NameError:
    pass

# Batch parameters
BATCH_SIZE   = 100
SHUFFLE      = False
RESUME       = True

# Which batch to start/stop from (1-based)
START_BATCH  = 31
END_BATCH    = None

# Create ZIP for each batch upon completion (for download convenience)
AUTO_ZIP_BATCH = True

# How often to flush manifests (every N files)
FLUSH_EVERY = 10

if SHUFFLE:
    random.seed(42); random.shuffle(all_paths)

total = len(all_paths)
num_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
start_b = max(START_BATCH - 1, 0)
end_b   = num_batches if END_BATCH is None else min(END_BATCH, num_batches)

print(f"Found {total} images under {INPUT_DIRS}")
print(f"Will run batches {start_b+1} .. {end_b} (of {num_batches})")

def open_manifest(path):
    write_header = not (os.path.exists(path) and os.path.getsize(path) > 0)
    f = open(path, "a" if not write_header else "w", newline="", encoding="utf-8-sig")
    return f, write_header

ts = datetime.now().strftime("%Y%m%d_%H%M%S")
root_manifest = os.path.join(DIRS["output"], f"manifest_{ts}.csv")
os.makedirs(DIRS["output"], exist_ok=True)

with open(root_manifest, "w", newline="", encoding="utf-8-sig") as mf:
    writer = csv.writer(mf)
    writer.writerow(["batch_id","idx_in_batch","input_path_rel","output_path_rel","n_masks","status"])

    for b in range(start_b, end_b):
        batch_id   = f"batch_{b+1:03d}"
        batch_dir  = os.path.join(DIRS["output"], batch_id)
        os.makedirs(batch_dir, exist_ok=True)

        # per-batch manifest (append if exists)
        batch_manifest = os.path.join(batch_dir, "manifest.csv")
        bf, need_header = open_manifest(batch_manifest)
        bwriter = csv.writer(bf)
        if need_header:
            bwriter.writerow(["idx_in_batch","input_path_rel","output_path_rel","n_masks","status"])

        s = b * BATCH_SIZE
        e = min(s + BATCH_SIZE, total)
        batch_paths = all_paths[s:e]
        print(f"\n=== {batch_id} ({s+1}-{e}/{total}) ===")

        for j, p in enumerate(batch_paths, 1):
            base     = os.path.splitext(os.path.basename(p))[0]
            out_path = os.path.join(batch_dir, f"{base}_overlay.png")

            status, n_masks = "ok", ""
            if RESUME and os.path.exists(out_path):
                status = "skipped_exists"
            else:
                img_bgr = cv2.imread(p, cv2.IMREAD_UNCHANGED)
                if img_bgr is None:
                    status = "read_error"
                else:
                    try:
                        img_rgb = to_rgb(img_bgr)
                        h, w    = img_rgb.shape[:2]
                        masks       = mask_generator.generate(img_rgb)
                        n_masks     = len(masks)
                        overlay_rgb = colorize_instances_sam2(masks, h, w)
                        blended     = blend_overlay(img_rgb, overlay_rgb, alpha=ALPHA)
                        cv2.imwrite(out_path, cv2.cvtColor(blended, cv2.COLOR_RGB2BGR))
                    except Exception as e:
                        status = f"error:{type(e).__name__}"
                        n_masks = ""

            rel_in  = os.path.relpath(p,        MAP_ROOT if 'MAP_ROOT' in globals() else DIRS["root"])
            rel_out = os.path.relpath(out_path, MAP_ROOT if 'MAP_ROOT' in globals() else DIRS["root"])
            row_root  = [batch_id, j, rel_in, rel_out, n_masks, status]
            row_batch = [j,       rel_in, rel_out, n_masks, status]
            writer.writerow(row_root)
            bwriter.writerow(row_batch)

            # small keep-alive output and manifest saving
            if (j % FLUSH_EVERY) == 0 or j == len(batch_paths):
                print(f"  progress: {j}/{len(batch_paths)}")
                mf.flush(); bf.flush()

        bf.close()

        # Automatic ZIP for this batch
        if AUTO_ZIP_BATCH:
            zip_path = batch_dir + ".zip"
            try:
                if os.path.exists(zip_path):
                    os.remove(zip_path)  # refresh if files were added
                shutil.make_archive(batch_dir, "zip", batch_dir)
                print(f"Zipped: {zip_path}")
            except Exception as e:
                print("ZIP error:", e)

print("\n✔️ Done.")
print("Root manifest:", root_manifest)
print("Per-batch manifests & ZIPs are under:", DIRS["output"])


In [ ]:
# !tar -C /kaggle -czf /kaggle/working/sem_sam2_output_$(date +%Y%m%d_%H%M%S).tgz working/SEM_SAM2/output
# !ls -lh /kaggle/working | grep tgz